In [1]:
import tensorflow as tf
import numpy as np
import data
import model as ml
from configs import DEFINES

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)
word2idx,  idx2word, vocabulary_length = data.load_vocabulary()

In [3]:
classifier = tf.estimator.Estimator(
        model_fn=ml.model,
        model_dir=DEFINES.check_point_path, 
        params={ 
            'hidden_size': DEFINES.hidden_size, 
            'layer_size': DEFINES.layer_size, 
            'learning_rate': DEFINES.learning_rate, 
            'vocabulary_length': vocabulary_length, 
            'embedding_size': DEFINES.embedding_size
        })

확률적 답변을 위해 다항분포 샘플링을 이용한다.<br>
preds는 디코더의 출력인 softmax이고, beta는 softmax의 강도를 조절하는 변수이다.<br>
beta가 작아질수록 동적으로 답변하고 (랜덤 성향이 큼), beta가 커질수록 정적으로 답변한다.

In [4]:
def pred_indices(preds, beta = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / beta
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

채팅 시작

In [5]:
print("\n채팅 시작:")
for i in range(10):
    question = input("Q: ")
    if question == 'quit':
        break
    
    predic_input_enc = data.data_processing([question], word2idx, DEFINES.enc_input)
    predic_output_dec = data.data_processing([""], word2idx, DEFINES.dec_input)
    predic_target_dec = data.data_processing([""], word2idx, DEFINES.dec_target)
    
    predictions = classifier.predict(input_fn=lambda:data.input_fn(
                        predic_input_enc,
                        predic_output_dec,
                        predic_target_dec,
                        1,
                        1))
    
    # 답변 문장에 대한 softmax 확률을 받는다.
    prob = np.array([v['indexs'] for v in predictions])
    prob = np.squeeze(prob)
    
    # 확률적으로 답변 문장의 인덱스를 생성한다.
    words_index = [pred_indices(p, beta = DEFINES.softmax_beta) for p in prob]

    # 답변 문장의 인덱스를 실제 문장으로 변환한다.
    answer = ""
    for word in words_index:
        if word !=0 and word !=2: # PAD = 0, END = 2
            answer += idx2word[word]
            answer += " "
            
    print("A:", answer)


채팅 시작:
Q: 안녕

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

A: 안녕하세요 드세요 
Q: 사랑해
A: 하늘 높나봅니다 
Q: 가끔 궁금해
A: 그 먼저 제가 
Q: 오늘 너무 심심해
A: 아파트가 예뻐요 
Q: 오늘 심심해
A: 친구들과 예뻐요 
Q: 심심해
A: 친구들과 예뻐요 
Q: 배 고프다
A: 서로 멋져요 예뻐요 
Q: 자연어 너무 어려워
A: 어느덧 잘 제가 
Q: quit
